In [20]:
## import auto reload
%load_ext autoreload
%autoreload 2

from simulation import run_simulation, initialize_simulation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
num_employees = 100
num_teams = 10
min_team_size = 8
num_friendships = 5
num_spaces = 20
space_types = ["Workstation"] * 10 + ["Meeting Room"] * 5 + ["Break Area"] * 3 + ["Quiet Area"] * 2
space_capacities = [8] * 10 + [15] * 5 + [20] * 3 + [10] * 2
hierarchy_levels = 5
hierarchy_adjustment_factor = 0.1

emps, spaces, team_graph, friendship_graph = initialize_simulation(num_employees, num_teams, min_team_size,
                                                                   num_friendships, num_spaces, space_types, space_capacities, hierarchy_levels, hierarchy_adjustment_factor)

run_simulation(emps, spaces, 10, initial_info_holder=1, team_graph=team_graph, friendship_graph=friendship_graph)


Time step 1
Informed employees: 10/100 (10.00%)
Space 1 (Workstation): [1, 9, 11, 21, 22, 25, 27, 28, 29, 31]
Space 2 (Workstation): [87, 92, 97, 100]
Space 3 (Workstation): [5, 7, 10, 12, 13, 14, 16, 18, 20, 30]
Space 4 (Workstation): [33, 36, 37, 38, 39, 41, 42, 43, 44, 46]
Space 5 (Workstation): [73, 81, 98]
Space 6 (Workstation): [2, 34, 35, 40, 45, 47, 49, 52, 53, 54]
Space 7 (Workstation): []
Space 8 (Workstation): []
Space 9 (Workstation): [58, 59, 61, 64, 65, 69, 70, 79, 80, 83]
Space 10 (Workstation): [3, 4, 6, 8, 15, 17, 19, 23, 24, 26]
Space 11 (Meeting Room): [50, 77]
Space 12 (Meeting Room): [71]
Space 13 (Meeting Room): [55]
Space 14 (Meeting Room): []
Space 15 (Meeting Room): [68, 76]
Space 16 (Break Area): [66, 78, 84]
Space 17 (Break Area): [62, 67, 93]
Space 18 (Break Area): [32, 48, 51, 56, 57, 63, 72, 74, 75, 82, 85, 86, 89, 90, 91, 95]
Space 19 (Quiet Area): []
Space 20 (Quiet Area): [60, 88, 94, 96, 99]
Information spread: [1, 9, 11, 21, 22, 25, 27, 28, 29, 31]

T